In [53]:
import torch 
from torchtext.datasets import IMDB
from torch.utils.data import DataLoader

In [2]:
train_iter = iter(IMDB(split='train'))

In [34]:
next(train_iter)

(2,
 "Zentropa is the most original movie I've seen in years. If you like unique thrillers that are influenced by film noir, then this is just the right cure for all of those Hollywood summer blockbusters clogging the theaters these days. Von Trier's follow-ups like Breaking the Waves have gotten more acclaim, but this is really his best work. It is flashy without being distracting and offers the perfect combination of suspense and dark humor. It's too bad he decided handheld cameras were the wave of the future. It's hard to say who talked him away from the style he exhibits here, but it's everyone's loss that he went into his heavily theoretical dogma direction instead.")

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [36]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator 

toker = get_tokenizer("basic_english")
train_iter = IMDB(split='train')

def yield_tokens(data_iter):
    for _,text in data_iter:
        yield toker(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=['unk'])
vocab.set_default_index(vocab['unk'])

In [63]:
vocab(toker("I like ice"))

[12, 44, 2065]

In [47]:
text_pipeline = lambda x: vocab(toker(x))
label_pipeline = lambda x: int(x) - 1

In [50]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [54]:
train_iter = IMDB(split="train")
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [64]:
num_class = len(set([label for (label,text) in train_iter]))

In [65]:
num_class

2

In [66]:
len(vocab)

100683

In [57]:
import torch.nn as nn 